In [1]:
!pip install -q langchain dspy-ai crewai google-search-results numexpr


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from langchain.agents import AgentExecutor, load_tools
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import Tool
from scripts.modified_calculator import LLMMathChain

MODEL_ID = "gpt-3.5-turbo"

llm = ChatOpenAI(model=MODEL_ID, temperature=0)

langchain_serpapi_tool = load_tools(["serpapi"], llm=llm)[0]
# Rename tools in the same format used by other tools

/home/ubuntu/venv/agents/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# DSPy

In [4]:
import dspy

gpt4_llm = dspy.OpenAI(model=MODEL_ID)
dspy.settings.configure(lm=gpt4_llm)

### Define tools

In [5]:
langchain_math_tool = Tool(
    name="Calculator",
    description="Useful for when you need to answer questions about math.",
    func=LLMMathChain.from_llm(llm=llm).run,
    coroutine=LLMMathChain.from_llm(llm=llm).arun,
)


class DSPyMathTool:
    name = "calculator"
    input_variable = "math formula"
    desc = "Useful for when you need to answer questions about math."

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return langchain_math_tool.run(*args, **kwargs)


class DSPySerpAPITool:
    name = "search"
    input_variable = "search input"
    desc = langchain_serpapi_tool.description

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return langchain_serpapi_tool.run(*args, **kwargs)

In [6]:
import datetime
import dspy


class GiveTime:
    name = "GiveTime"
    input_variable = "empty"
    desc = "takes an empty string and returns the current local time"

    def __init__(self, k=3):
        pass

    def __call__(self, *args, **kwargs):
        return datetime.datetime.now().strftime("%H:%M:%S")

In [7]:
math_tool = DSPyMathTool()
serpapi_tool = DSPySerpAPITool()
agent_dspy = dspy.ReAct("question -> answer", tools=[math_tool, serpapi_tool])

In [8]:
async def call_dspy(agent, question: str) -> str:
    response = {"output": agent(question=question).answer, "intermediate_steps": []}
    return response

In [9]:
await call_dspy(agent_dspy, "what is 2+2")

{'output': '4', 'intermediate_steps': []}

In [10]:
gpt4_llm.inspect_history()





You will be given `question` and you will respond with `answer`.

To do this, you will interleave Thought, Action, and Observation steps.

Thought can reason about the current situation, and Action can be the following types:

(1) calculator[math formula], which Useful for when you need to answer questions about math.
(2) search[search input], which A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
(3) Finish[answer], which returns the final `answer` and finishes the task

---

Follow the following format.

Question: ${question}

Thought 1: next steps to take based on last observation

Action 1: always either calculator[math formula] or search[search input] or, when done, Finish[answer]

Observation 1: observations based on action

Thought 2: next steps to take based on last observation

Action 2: always either calculator[math formula] or search[search input] or, when done, Finish[answer]

Observation 2: observations

### Compile DSPy

In [10]:
import datasets

math_dataset = datasets.load_dataset("gsm8k", "main")["train"]


def build_dspy_trainset(dataset):
    dspy_dataset = []
    for i in range(len(dataset)):
        dspy_dataset.append(
            dspy.Example(
                question=dataset["question"][i],
                answer=dataset["answer"][i],
            ).with_inputs("question")
        )
    return dspy_dataset


# from dspy.teleprompt import BootstrapFewShot

# teleprompter = BootstrapFewShot(metric=validate_with_llm_judge, max_rounds=5)
# compiled_rag = teleprompter.compile(
#     agent,
#     trainset=build_dspy_trainset(math_dataset[:20]),
#     valset=build_dspy_trainset(math_dataset[20:50]),
# )

# Benchmark different frameworks

In [11]:
math_dataset = (
    datasets.load_dataset("reasoning-machines/gsm-hard", "main")["train"]
    .shuffle(seed=42)
    .select(range(30))
)
math_dataset = math_dataset.rename_column("input", "question")
math_dataset = math_dataset.rename_column("target", "answer")

task_column = ["GSM8K-hard"] * len(math_dataset)
math_dataset = math_dataset.add_column("task", task_column).select_columns(
    ["question", "answer", "task"]
)
eval_dataset = math_dataset

/home/ubuntu/venv/agents/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


In [14]:
eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

from scripts.run_agents import answer_questions

### DSPy

In [14]:
# DSPy
await answer_questions(
    eval_dataset,
    agent_dspy,
    "dspy_agent",
    output_folder="output_dspy2",
    agent_call_function=call_dspy,
)

  0%|          | 0/30 [00:00<?, ?it/s]

 13%|█▎        | 4/30 [01:00<06:38, 15.31s/it]

### Vanilla GPT

In [15]:
prompt = "Answer the question below. You can think out loud, but remind to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. Question:"


async def call_vanilla_gpt(agent, question: str):
    output = agent.invoke(prompt + question).content
    if "Final Answer:" in output:
        output = output.split("Final Answer:")[1]
    response = {"output": output, "intermediate_steps": []}
    return response

In [16]:
llm_gpt = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

await answer_questions(
    eval_dataset,
    llm_gpt,
    "gpt3.5_agent_notools",
    output_folder="output_dspy2",
    agent_call_function=call_vanilla_gpt,
)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:43<00:00,  1.44s/it]


[{'agent_name': 'gpt3.5_agent_notools',
  'question': 'Charlie wants to sell beeswax candles.  For every pound of beeswax, he can make 10 tapered candles.  One pound of beeswax and the wicks cost $10.00 in supplies.   If he sells each candle for $2.00 each, what is his net profit if he makes and sells 1080379 candles?',
  'gt_answer': 1080379.0,
  'prediction': '` $1080379.',
  'intermediate_steps': [],
  'parsing_error': False,
  'iteration_limit_exceeded': False,
  'agent_error': None,
  'start_time': '2024-02-07 15:56:55',
  'end_time': '2024-02-07 15:56:57',
  'task': 'GSM8K-hard'},
 {'agent_name': 'gpt3.5_agent_notools',
  'question': 'Melissa is summoned to jury duty. She spends 6 hours a day for 3 days listening to a court case. If Melissa is paid $557654 per day but also has to pay $3 for parking each day, how much jury pay does she make per hour after expenses?',
  'gt_answer': 92941.8333333333,
  'prediction': '` Melissa makes $30980.61 per hour after expenses.',
  'intermedi

### LangChain

In [17]:
from scripts.agents import build_openai_agent

langchain_agent = build_openai_agent(model_id="gpt-3.5-turbo-1106")

await answer_questions(
    eval_dataset,
    langchain_agent,
    "gpt3.5_langchain2",
    output_folder="output_dspy2",
)

/home/ubuntu/venv/agents/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(
  0%|          | 0/30 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...

Invoking: `calculator` with `10*1080379*2-(10.00+1080379)`


20527191.0

  3%|▎         | 1/30 [00:04<02:06,  4.37s/it]

Charlie's net profit, if he makes and sells 1,080,379 candles, would be $20,527,191.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `calculator` with `(557654 - 3) / (6 * 3)`


30980.61111111111

  7%|▋         | 2/30 [00:09<02:14,  4.82s/it]

...

 97%|█████████▋| 29/30 [13:59<00:25, 25.96s/it]

Peter will have a total of approximately 476,547 action figures by the end of the week.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `calculator` with `(40*10 + 4896852*12) / 2`


29381312.0

100%|██████████| 30/30 [14:12<00:00, 28.41s/it]

The average total distance spat by Zoey and Sydney is 29,381,312 feet.

> Finished chain.


[{'agent_name': 'gpt3.5_langchain2',
  'question': 'Charlie wants to sell beeswax candles.  For every pound of beeswax, he can make 10 tapered candles.  One pound of beeswax and the wicks cost $10.00 in supplies.   If he sells each candle for $2.00 each, what is his net profit if he makes and sells 1080379 candles?',
  'gt_answer': 1080379.0,
  'prediction': "Charlie's net profit, if he makes and sells 1,080,379 candles, would be $20,527,191.",
  'intermediate_steps': [{'tool': 'calculator',
    'tool_input': '10*1080379*2-(10.00+1080379)',
    'tool_output': '20527191.0'}],
  'parsing_error': False,
  'iteration_limit_exceeded': False,
  'agent_error': None,
  'start_time': '2024-02-07 15:57:38',
  'end_time': '2024-02-07 15:57:43',
  'task': 'GSM8K-hard'},
 {'agent_name': 'gpt3.5_langchain2',
  'question': 'Melissa is summoned to jury duty. She spends 6 hours a day for 3 days listening to a court case. If Melissa is paid $557654 per day but also has to pay $3 for parking each day, ho

### CrewAI

In [ ]:
from crewai import Agent, Task, Crew, Process


async def call_crew_sequential(agent, question: str):
    output = agent.invoke(prompt + question).content
    # Creating a senior researcher agent
    researcher = Agent(
        role="Senior Researcher",
        goal=f"Solve this question: {question}",
        verbose=True,
        backstory="""Driven by curiosity, you like solving math problems""",
    )

    # Creating a writer agent
    veryfier = Agent(
        role="Senior Verifyer",
        goal=f"Check results of others for this question: {question}",
        verbose=True,
        backstory="""You have a flair to detect errors in math problems and correct them.""",
    )
    # Research task for identifying AI trends
    solving_task = Task(
        description=f"""Solve the given question: {question}. Think step-by-step.""",
        expected_output="A short answer",
        max_inter=3,
        tools=[langchain_math_tool, langchain_serpapi_tool],
        agent=researcher,
    )

    # Writing task based on research findings
    verification_task = Task(
        description=f"""You have to answer {question}, then provide a final answer.
        Think step-by-step to correct any errors. Remember to preface your final answer with 'Final Answer:'. In your final answer, do not talk about wether the previous answer was correct or not. Just provide the corrected answer.
        """,
        expected_output=f"The corrected answer to the question {question}",
        tools=[langchain_math_tool, langchain_serpapi_tool],
        agent=veryfier,
    )
    # Forming the tech-focused crew
    crew = Crew(
        agents=[researcher, veryfier],
        tasks=[solving_task, verification_task],
        process=Process.sequential,  # Sequential task execution
    )
    output = crew.kickoff()

    if "Final Answer:" in output:
        output = output.split("Final Answer:")[1]
    response = {"output": output, "intermediate_steps": []}
    return response


llm_gpt = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

await answer_questions(
    eval_dataset,
    llm_gpt,
    "gpt3.5_crewai_sequential",
    output_folder="output_dspy",
    agent_call_function=call_crew_sequential,
)

100%|██████████| 30/30 [00:00<00:00, 8927.22it/s]


[{'agent_name': 'gpt3.5_crewai_sequential',
  'question': 'A craft store makes a third of its sales in the fabric section, a quarter of its sales in the jewelry section, and the rest in the stationery section. They made 36 sales today. How many sales were in the stationery section?',
  'gt_answer': 'The craft store made 36 / 3 = <<36/3=12>>12 sales in the fabric section.\nIt made 36 / 4 = <<36/4=9>>9 sales in the jewelry section.\nThus, there were 36 - 12 - 9 = <<36-12-9=15>>15 sales in the stationery section.\n#### 15',
  'prediction': 'The stationery section made 15 sales.',
  'intermediate_steps': [],
  'parsing_error': False,
  'iteration_limit_exceeded': False,
  'agent_error': None,
  'start_time': '2024-02-07 14:35:10',
  'end_time': '2024-02-07 14:35:50',
  'task': 'GSM8K'},
 {'agent_name': 'gpt3.5_crewai_sequential',
  'question': "Marcy is a makeup artist and has agreed to do some makeup for her friend's wedding. The only makeup she has limited stock of is lip gloss so she co

# Evaluate results

In [18]:
import pandas as pd
import glob
import json

result_df = pd.concat([pd.read_json(f) for f in glob.glob("output_dspy2/*.json")])
result_df = result_df.drop(["start_time", "end_time"], axis=1)
answers = result_df.to_dict(orient="records")
with open("output/results_dspy_experiment.json", "w") as f:
    json.dump(answers, f)

In [19]:
from scripts.evaluation import evaluate_answers
from scripts.prompts import EVALUATION_PROMPT_TEMPLATE

output_file_name = "output/results_dspy_experiment.json"

with open(output_file_name, "r") as f:
    answers = json.load(f)

results = await evaluate_answers(
    answers,
    eval_chat_model,
    "GPT4",
    EVALUATION_PROMPT_TEMPLATE,
    output_file=output_file_name,
)
result_df = pd.DataFrame.from_records(results)

print("Evaluation is complete!")

  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 64/64 [00:20<00:00,  3.20it/s]

Evaluation is complete!


In [20]:
result_df["eval_score_GPT4"] = result_df["eval_score_GPT4"].astype(float)
result_df["eval_score_GPT4"] = (result_df["eval_score_GPT4"] - 1) / 4
display(result_df.groupby("agent_name")["eval_score_GPT4"].mean())

agent_name
dspy_agent              0.000000
gpt3.5_agent_notools    0.566667
gpt3.5_langchain2       0.433333
Name: eval_score_GPT4, dtype: float64

##### Thoughts on DSPy:
- Contrary to the framework's promise (see discussions [here](https://news.ycombinator.com/item?id=37417698)), many prompts are hardcoded: [example](https://github.com/search?q=repo%3Astanfordnlp%2Fdspy%20To%20do%20this%2C%20you%20will%20interleave%20Thought%2C%20Action%2C%20and%20Observation%20steps.&type=code)
- The frameworks makes it really complicated to get visibility on what's happening under the hood.
- Probably mainly for this reason, I did not manage to get it to achieve correct performance in 1 day of work.

# CrewAI